In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow 
import time
#pip install -U scikit-learn scipy matplotlib
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import re

from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras import models, layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow_model_optimization.sparsity import keras as sparsity

In [ ]:
def load_mnist():
    #Load dataset as train and test sets
    global x_test,x_train,y_test,y_train,mnist_ds
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(60000, 784)
    x_test = x_test.reshape(10000, 784)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    num_classes = 10
    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    mnist_train, _ = tf.keras.datasets.mnist.load_data()
    images = tf.cast(mnist_train[0], tf.float32) / 255.0
    mnist_ds = tf.data.Dataset.from_tensor_slices((images)).batch(1)

In [ ]:
def define_model():
    from keras.models import Sequential
    from keras import models, layers
    from keras import regularizers
    model = keras.Sequential()
    model.add(keras.layers.Dropout(0.2,input_shape=(784,)))
    model.add(keras.layers.Dense(1000,
                            kernel_regularizer = regularizers.l2(0.01),
                            activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(1000,
                            kernel_regularizer = regularizers.l2(0.01),
                            activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(10,  activation='softmax'))
    #display the model summary
    model.summary()

In [ ]:
def train_mode(model):
    model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer='adam', 
              metrics=['accuracy'])
    hist = model.fit(x_train, y_train,
                        batch_size=128,
                        epochs=10,
                        verbose=1,
                        validation_data=(x_test,y_test))
    #Save the entire model in model.h5 file
    model.save("trained_model.h5")
    print("Saved model to disk")

In [ ]:
def load_model(m):
    model = tf.keras.models.load_model(m)
    return model

In [ ]:
def representative_data_gen():
  for input_value in mnist_ds.take(100):
    yield [input_value]

In [ ]:
quantize_layer_names1 = ['dense_1']
def convert_model_1(model,name='converted_quant_int_model1.tflite',quantize_annotations = quantize_layer_names1,partial=True):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_data_gen
    if partial:
        converter.quantize_annotations = quantize_annotations
    tflite_quant_int_model = converter.convert()
    #saving converted model in "converted_quant_model.tflite" file
    open(name, "wb").write(tflite_quant_int_model)

In [ ]:

def just_convert_model_1(model,name='just_converted_quant_int_model1.tflite'):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_quant_int_model = converter.convert()
    #saving converted model in "converted_quant_model.tflite" file
    open(name, "wb").write(tflite_quant_int_model)

In [ ]:
import os
def prepare_dataset():
    # Load the training data into a tf.data.Dataset
    train_images_dir = "train2017"

    def load_images(images_dir=train_images_dir):
        images = []
        labels = []
        for image_filename in os.listdir(images_dir):
            # Load the image
            image = tf.io.read_file(os.path.join(images_dir, image_filename))
            image = tf.image.decode_jpeg(image, channels=3)

            # Load the label
            #label = # code to load label from file

            images.append(image)
            #labels.append(label)
        return images
        #, labels

    images = load_images(train_images_dir)
    train_dataset = tf.data.Dataset.from_tensor_slices((images))
    # Generate a representative dataset
    return train_dataset

def gen_rep():
    train_dataset=prepare_dataset()    
    representative_dataset = train_dataset.take(100).batch(1)
    return representative_dataset

In [ ]:
'''for layer_name in quantize_layer_names:
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.uint8
    converter.inference_output_type = tf.uint8
    input_arrays = converter.get_input_arrays()
    for input_array in input_arrays:
        converter.quantizers[input_array].supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.quantizers[layer_name].supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
'''
quantize_layer_names2 = ['dense_1']
def convert_model_2(model,name='converted_quant_int_model2.tflite',quantize_annotations = quantize_layer_names2,partial=True):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    #converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = gen_rep()
    if partial:
        converter.quantize_annotations = quantize_annotations
    tflite_quant_int_model = converter.convert()
    #saving converted model in "converted_quant_model.tflite" file
    open(name, "wb").write(tflite_quant_int_model)

In [ ]:
def evaluate1(name='converted_quant_int_model1.tflite'):
    # Load TFLite model and allocate tensors.
    interpreter = \
        tf.lite.Interpreter(model_path=name)
    interpreter.allocate_tensors()
    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    # Test model on some input data.
    input_shape = input_details[0]['shape']
    acc=0
    for i in range(len(x_test)):
        input_data = x_test[i].reshape(input_shape)
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        if(np.argmax(output_data) == np.argmax(y_test[i])):
            acc+=1
    acc = acc/len(x_test)
    print(acc*100)

In [4]:
model=load_model(m='Yolov3.h5')
model.summary()
just_convert_model_1(model)
